In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 77.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyvi import ViTokenizer
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import time
from keras.utils import set_random_seed
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_x = pd.read_csv('/content/drive/MyDrive/CV/Student_feedback/UIT-VSFC/train/feedback.txt', sep=' \n', engine='python', header=None, names=['feedback'])
train_y =  pd.read_csv('/content/drive/MyDrive/CV/Student_feedback/UIT-VSFC/train/label.txt', sep=" \n", engine='python', header=None, names=['label'])

test_x = pd.read_csv('/content/drive/MyDrive/CV/Student_feedback/UIT-VSFC/test/feedback.txt', sep=' \n', engine='python', header=None, names=['feedback'])
test_y =  pd.read_csv('/content/drive/MyDrive/CV/Student_feedback/UIT-VSFC/test/label.txt', sep=" \n", engine='python', header=None, names=['label'])

In [5]:
train_x

,feedback
0,slide giáo trình đầy đủ .
1,"nhiệt tình giảng dạy , gần gũi với sinh viên ."
2,đi học đầy đủ full điểm chuyên cần .
3,chưa áp dụng công nghệ thông tin và các thiết ...
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng..."
...,...
13004,hướng dẫn lab mơ hồ .
13005,thầy cho chúng em những bài tập mang tính thực...
13006,thầy không dạy nhiều chủ yếu cho sinh viên tự ...
13007,em muốn đổi tên môn học vì tên môn là lập trìn...


In [6]:
import re
stopword_list = pd.read_csv('/content/drive/MyDrive/CV/Student_feedback/stopword_vietnamese.csv', header=None)
stopword_list = list(stopword_list[0])

def Preprocessing(feedback):
    # xóa ký tự kéo dài
    feedback = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), feedback, flags=re.I)

    # chuẩn hóa chữ thường
    feedback = feedback.lower()

    # xóa ký hiệu teencode
    icons = [':)', ':(', '@@', '<3', ':d', ':3', ':v', ':_', ':p', '>>', ':">', '^^', 'v.v', ':B', ':^', ':v', 'y_y', 'u_u']
    for icon in icons:
        feedback = feedback.replace(icon,' ')

    # xóa từ viết tắt mà tác giả đã biến đổi
    acronyms = ['colon', 'smile', 'sad', 'surprise', 'love', 'contemn', 'big', 'smile', 'cc',
                'small', 'hihi', 'double', 'vdotv','dot', 'fraction', 'csharp']
    for acronym in acronyms:
        feedback = feedback.replace(acronym, ' ')

    # xóa ký tự đặc biệt
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for punc in punctuation:
        feedback = feedback.replace(punc,' ')

    #feedback = ' '.join([cmt for cmt in feedback.split() if cmt not in stopword_list])

    # ViTokenize tách từ
    feedback = ViTokenizer.tokenize(feedback)

    return feedback

In [7]:
for i in tqdm(range(len(train_x['feedback']))):
    train_x['feedback'][i] = Preprocessing(train_x['feedback'][i])

for i in tqdm(range(len(test_x['feedback']))):
    test_x['feedback'][i] = Preprocessing(test_x['feedback'][i])

100%|██████████| 3166/3166 [00:00<00:00, 3341.32it/s]


In [8]:
X_train = np.array(train_x['feedback'])
X_test = np.array(test_x['feedback'])

In [9]:
vocab_size = 3915
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok, filters='')

In [10]:
tokenizer.fit_on_texts(X_train)
print("Number of Documents: ", tokenizer.document_count)
print("Number of Words: ", tokenizer.num_words)

Number of Documents:  13009
Number of Words:  3915


In [11]:
len(tokenizer.index_word.values())

3915

In [12]:
tokenizer.index_word

{1: '<OOV>',
 2: 'thầy',
 3: 'sinh_viên',
 4: 'dạy',
 5: 'và',
 6: 'không',
 7: 'rất',
 8: 'nhiệt_tình',
 9: 'cho',
 10: 'có',
 11: 'hiểu',
 12: 'nhiều',
 13: 'giảng_viên',
 14: 'học',
 15: 'dễ',
 16: 'cô',
 17: 'bài',
 18: 'giảng',
 19: 'nên',
 20: 'bài_tập',
 21: 'được',
 22: 'hơn',
 23: 'em',
 24: 'với',
 25: 'giảng_dạy',
 26: 'của',
 27: 'hay',
 28: 'kiến_thức',
 29: 'các',
 30: 'thực_hành',
 31: 'trong',
 32: 'để',
 33: 'cần',
 34: 'tốt',
 35: 'làm',
 36: 'môn_học',
 37: 'quá',
 38: 'là',
 39: 'về',
 40: 'chưa',
 41: 'khó',
 42: 'tận_tâm',
 43: 'lớp',
 44: 'những',
 45: 'giờ',
 46: 'truyền_đạt',
 47: 'giáo_viên',
 48: 'khi',
 49: 'cách',
 50: 'nhưng',
 51: 'đến',
 52: 'hơi',
 53: 'còn',
 54: 'thời_gian',
 55: 'thêm',
 56: 'thì',
 57: 'tận_tình',
 58: 'trên',
 59: 'nội_dung',
 60: 'lý_thuyết',
 61: 'môn',
 62: 'phòng',
 63: 'nói',
 64: 'chỉ',
 65: 'này',
 66: 'một',
 67: 'cũng',
 68: 'vào',
 69: 'lại',
 70: 'hướng_dẫn',
 71: 'ra',
 72: 'mà',
 73: 'việc',
 74: 'khá',
 75: 'vui_tính'

In [13]:
train_sequences = tokenizer.texts_to_sequences(X_train)

In [14]:
tokenizer.texts_to_sequences(['slide giáo_trình đầy_đủ'])

[[84, 202, 96]]

In [15]:
max = 0
sum = 0
for x in train_sequences:
  sum += len(x)
  if len(x) > max:
    max = len(x)
print('Câu có độ dài lớn nhất: ', max)
print('Độ dài trung bình câu: ', sum/len(train_sequences))

Câu có độ dài lớn nhất:  123
Độ dài trung bình câu:  9.505342455223307


In [16]:
sequence_length = 123
train_padded = pad_sequences(train_sequences, maxlen=sequence_length, padding='post', truncating='post')

In [17]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=sequence_length, padding='post', truncating='post')

In [18]:
# encode = OneHotEncoder(sparse_output=False)
# y_train = encode.fit_transform(train_y[['label']])
# y_test = encode.transform(test_y[['label']])

In [19]:
y_train = train_y['label']
y_test = test_y['label']

In [20]:
#from imblearn.over_sampling import SMOTE, KMeansSMOTE, BorderlineSMOTE, SVMSMOTE
# from imblearn.over_sampling import ADASYN
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.svm import SVC
# #smote = SVMSMOTE(sampling_strategy='minority', random_state=42, svm_estimator=SVC(C=10, gamma=0.1, kernel='rbf', random_state=42))
# #smote = SMOTE(k_neighbors=7, random_state=42)
#smote = BorderlineSMOTE(k_neighbors=7, random_state=42, kind='borderline-2')    # f1 = 76.36
# #smote = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=42)
#X_train, y_train = smote.fit_resample(train_padded, y_train)

In [21]:
X_train = train_padded
X_test = test_padded

In [22]:
X_train.shape

(13009, 123)

In [23]:
X_test.shape

(3166, 123)

In [24]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [25]:
from keras.utils import set_random_seed
set_random_seed(42)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=150, input_length=sequence_length))
#model.add(Conv1D(filters=128, kernel_size=7, activation='relu'))
#model.add(AveragePooling1D(pool_size=2))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(10, activation='tanh'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[f1_m])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 123, 150)          587250    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               46848     
 l)                                                              
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 634,781
Trainable params: 634,781
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=1, callbacks=callback, shuffle=False)

Epoch 1/100
407/407 [==============================] - 46s 85ms/step - loss: 0.3849 - f1_m: 0.9039 - val_loss: 0.3349 - val_f1_m: 0.7476
Epoch 2/100
407/407 [==============================] - 6s 14ms/step - loss: 0.2293 - f1_m: 0.7253 - val_loss: 0.3217 - val_f1_m: 0.7449
Epoch 3/100
407/407 [==============================] - 5s 13ms/step - loss: 0.1768 - f1_m: 0.7209 - val_loss: 0.3203 - val_f1_m: 0.7394
Epoch 4/100
407/407 [==============================] - 7s 18ms/step - loss: 0.1419 - f1_m: 0.7127 - val_loss: 0.3536 - val_f1_m: 0.7360
Epoch 5/100
407/407 [==============================] - 5s 13ms/step - loss: 0.1186 - f1_m: 0.7088 - val_loss: 0.3709 - val_f1_m: 0.7406
Epoch 6/100
407/407 [==============================] - 6s 16ms/step - loss: 0.0990 - f1_m: 0.7060 - val_loss: 0.3793 - val_f1_m: 0.7345
Epoch 7/100
407/407 [==============================] - 5s 13ms/step - loss: 0.0870 - f1_m: 0.7060 - val_loss: 0.3863 - val_f1_m: 0.7368
Epoch 8/100
407/407 [==========================

In [28]:
set_random_seed(42)

In [29]:
y_pred = model.predict(X_test)
y_pred = [x.argmax() for x in y_pred]
print(classification_report(y_test, y_pred, digits=4))
#0.7721

99/99 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0     0.8932    0.9496    0.9205      1409
           1     0.6238    0.3772    0.4701       167
           2     0.9324    0.9189    0.9256      1590

    accuracy                         0.9040      3166
   macro avg     0.8164    0.7486    0.7721      3166
weighted avg     0.8986    0.9040    0.8993      3166



In [30]:
# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [31]:
# from keras.models import load_model
# model_saved = load_model('my_model.h5')

# y_pred = model_saved.predict(X_test)
# y_pred = [x.argmax() for x in y_pred]
# print(classification_report(y_test, y_pred, digits=4))